In [1]:
%run -m ipy_startup
%run -m ipy_logging
%matplotlib inline
from musc_genomics.data_import import cosmic, api, prep
from musc_genomics import data
pd.set_option('max_info_rows', int(1E9))

In [2]:
path = os.path.expanduser('~/data/research/musc_genomics/materialized/cosmic_manova_input.csv')
d = cosmic.get_raw_cosmic_data(path)
d.info()

/Users/eczech/anaconda/envs/research3.5/lib/python3.5/site-packages/ipykernel/__main__.py:2: PerformanceWarning: indexing past lexsort depth may impact performance.
  from ipykernel import kernelapp as app


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 707 entries, 0 to 706
Columns: 2035 entries, (Cell Line, Unnamed: 0_level_1, Unnamed: 0_level_2, Unnamed: 0_level_3, Unnamed: 0_level_4, Unnamed: 0_level_5) to (AG-014699_IC_50_HIGH, 1175_IC_50_HIGH, PARP1/2_IC_50_HIGH, None required_IC_50_HIGH, Y_IC_50_HIGH, 0.01953125_IC_50_HIGH)
dtypes: float64(1960), int64(2), object(73)
memory usage: 11.0+ MB


In [3]:
pd.Series(d.columns.levels[1]).str.startswith('Unnamed:').value_counts()

False    1680
True       75
dtype: int64

In [4]:
d.head()

,Cell Line,Cosmic_ID,Cancer Type,Tissue,MS-HL,AKT2,ALK,APC,BRAF,BRCA1,...,JNJ-26854165_IC_50_HIGH,TW 37_IC_50_LOW,TW 37_IC_50,TW 37_IC_50_HIGH,CCT018159_IC_50_LOW,CCT018159_IC_50,CCT018159_IC_50_HIGH,AG-014699_IC_50_LOW,AG-014699_IC_50,AG-014699_IC_50_HIGH
,Unnamed: 0_level_1,Unnamed: 1_level_1,Unnamed: 2_level_1,Unnamed: 3_level_1,Unnamed: 4_level_1,Unnamed: 5_level_1,Unnamed: 6_level_1,Unnamed: 7_level_1,Unnamed: 8_level_1,Unnamed: 9_level_1,...,1133_IC_50_HIGH,1149_IC_50_LOW,1149_IC_50,1149_IC_50_HIGH,1170_IC_50_LOW,1170_IC_50,1170_IC_50_HIGH,1175_IC_50_LOW,1175_IC_50,1175_IC_50_HIGH
,Unnamed: 0_level_2,Unnamed: 1_level_2,Unnamed: 2_level_2,Unnamed: 3_level_2,Unnamed: 4_level_2,Unnamed: 5_level_2,Unnamed: 6_level_2,Unnamed: 7_level_2,Unnamed: 8_level_2,Unnamed: 9_level_2,...,MDM2_IC_50_HIGH,BCL-2 : BCL-XL_IC_50_LOW,BCL-2 : BCL-XL_IC_50,BCL-2 : BCL-XL_IC_50_HIGH,HSP90_IC_50_LOW,HSP90_IC_50,HSP90_IC_50_HIGH,PARP1/2_IC_50_LOW,PARP1/2_IC_50,PARP1/2_IC_50_HIGH
,Unnamed: 0_level_3,Unnamed: 1_level_3,Unnamed: 2_level_3,Unnamed: 3_level_3,Unnamed: 4_level_3,Unnamed: 5_level_3,Unnamed: 6_level_3,Unnamed: 7_level_3,Unnamed: 8_level_3,Unnamed: 9_level_3,...,None required_IC_50_HIGH,None required_IC_50_LOW,None required_IC_50,None required_IC_50_HIGH,None required_IC_50_LOW,None required_IC_50,None required_IC_50_HIGH,None required_IC_50_LOW,None required_IC_50,None required_IC_50_HIGH
,Unnamed: 0_level_4,Unnamed: 1_level_4,Unnamed: 2_level_4,Unnamed: 3_level_4,Unnamed: 4_level_4,Unnamed: 5_level_4,Unnamed: 6_level_4,Unnamed: 7_level_4,Unnamed: 8_level_4,Unnamed: 9_level_4,...,Y_IC_50_HIGH,Y_IC_50_LOW,Y_IC_50,Y_IC_50_HIGH,Y_IC_50_LOW,Y_IC_50,Y_IC_50_HIGH,Y_IC_50_LOW,Y_IC_50,Y_IC_50_HIGH
,Unnamed: 0_level_5,Unnamed: 1_level_5,Unnamed: 2_level_5,Unnamed: 3_level_5,Unnamed: 4_level_5,Unnamed: 5_level_5,Unnamed: 6_level_5,Unnamed: 7_level_5,Unnamed: 8_level_5,Unnamed: 9_level_5,...,0.0390625_IC_50_HIGH,0.01953125_IC_50_LOW,0.01953125_IC_50,0.01953125_IC_50_HIGH,0.0390625_IC_50_LOW,0.0390625_IC_50,0.0390625_IC_50_HIGH,0.01953125_IC_50_LOW,0.01953125_IC_50,0.01953125_IC_50_HIGH
0,MC-CAR,683665,blood,haematopoietic_neoplasm_other,0,na::0<cn<8,wt::0<cn<8,wt::0<cn<8,wt::0<cn<8,wt::0<cn<8,...,6.216977,-2.241483,-1.975136,-1.723958,1.513655,1.805149,2.160164,2.254131,3.573990,6.310013
1,PFSK-1,683667,nervous_system,medulloblastoma,0,na::0<cn<8,wt::0<cn<8,wt::0<cn<8,wt::0<cn<8,wt::0<cn<8,...,7.071140,-2.178437,-1.733143,-1.295565,2.181652,3.375968,6.420351,2.348972,4.627090,8.660155
2,A673,684052,soft_tissue,rhabdomyosarcoma,0,na::0<cn<8,wt::0<cn<8,wt::0<cn<8,p.V600E::0<cn<8,wt::0<cn<8,...,4.731629,-5.272234,-2.921642,-0.845975,1.981498,2.208358,2.596048,1.368350,1.678196,2.149977
3,ES3,684055,bone,ewings_sarcoma,0,na::0<cn<8,wt::0<cn<8,wt::0<cn<8,wt::0<cn<8,wt::0<cn<8,...,3.801544,-1.462394,-1.025733,-0.580209,1.787129,2.557677,4.722587,0.782539,1.246618,1.972593
4,ES5,684057,bone,ewings_sarcoma,0,na::0<cn<8,wt::0<cn<8,wt::0<cn<8,wt::0<cn<8,wt::0<cn<8,...,4.022325,-1.215029,-0.122957,0.739059,2.126826,2.493732,4.045132,1.384828,1.780458,2.372001


In [5]:
# Select fields with relevant drug info (there are a bunch of non-metadata 
# fields that appear to have no useful data on the left side of the file)
# by looking for those with "Unnamed" in the second column index level
mask = [not c[1].startswith('Unnamed:') for c in d.columns.values]
assert len(mask) == d.shape[1]

# Extract fields specific to drugs, or metadata in the left-most fields
drug_idx = list(np.sort(np.argwhere(mask)[:,0]))
d_sub = d.iloc[:, [0, 1, 2, 3] + drug_idx]

# Drop columns that have completely duplicated values 
# (otherwise indexing to them doesn't work when not unique)
d_sub = d_sub.T.drop_duplicates().T

# Collapse multi-index columns by joining values on "|", and ignoring 
# unnamed column level values 
def get_col(c):
    c = [x for x in c if not x.startswith('Unnamed:')]
    return '|'.join(c).replace(' ', '_')
d_sub.columns = [get_col(c) for c in d_sub]
    
# Rename upper underscore fields to upper camel case
d_sub = d_sub.rename(columns={
    'Cell_Line': 'CellLine',
    'Cosmic_ID': 'CosmicID',
    'Cancer_Type': 'CancerType'
})

# Make sure text transformations/renames did not introduce collisions
assert len(np.unique(d_sub.columns.tolist())) == len(d_sub.columns), \
    'Found duplicate column names after text cleanse'

d_sub.head()

,CellLine,CosmicID,CancerType,Tissue,Erlotinib_IC_50|1_IC_50|EGFR_IC_50|None_required_IC_50|Y_IC_50|0.0078125_IC_50,Rapamycin_IC_50|3_IC_50|mTOR_IC_50|None_required_IC_50|Y_IC_50|0.000390625_IC_50,Sunitinib_IC_50|5_IC_50|PDGFRA_:__PDGFRB_:__KDR_:__KIT_:__FLT3_IC_50|None_required_IC_50|Y_IC_50|0.03125_IC_50,PHA-665752_IC_50|6_IC_50|MET_IC_50|None_required_IC_50|Y_IC_50|0.0078125_IC_50,MG-132_IC_50|9_IC_50|Proteosome_IC_50|None_required_IC_50|Y_IC_50|0.00390625_IC_50,Paclitaxel_IC_50|11_IC_50|Microtubules_IC_50|None_required_IC_50|Y_IC_50|0.0004_IC_50,...,PF-4708671_IC_50_LOW|1129_IC_50_LOW|p70_S6KA_IC_50_LOW|None_required_IC_50_LOW|Y_IC_50_LOW|0.0390625_IC_50_LOW,PF-4708671_IC_50_HIGH|1129_IC_50_HIGH|p70_S6KA_IC_50_HIGH|None_required_IC_50_HIGH|Y_IC_50_HIGH|0.0390625_IC_50_HIGH,JNJ-26854165_IC_50_LOW|1133_IC_50_LOW|MDM2_IC_50_LOW|None_required_IC_50_LOW|Y_IC_50_LOW|0.0390625_IC_50_LOW,JNJ-26854165_IC_50_HIGH|1133_IC_50_HIGH|MDM2_IC_50_HIGH|None_required_IC_50_HIGH|Y_IC_50_HIGH|0.0390625_IC_50_HIGH,TW_37_IC_50_LOW|1149_IC_50_LOW|BCL-2_:__BCL-XL_IC_50_LOW|None_required_IC_50_LOW|Y_IC_50_LOW|0.01953125_IC_50_LOW,TW_37_IC_50_HIGH|1149_IC_50_HIGH|BCL-2_:__BCL-XL_IC_50_HIGH|None_required_IC_50_HIGH|Y_IC_50_HIGH|0.01953125_IC_50_HIGH,CCT018159_IC_50_LOW|1170_IC_50_LOW|HSP90_IC_50_LOW|None_required_IC_50_LOW|Y_IC_50_LOW|0.0390625_IC_50_LOW,CCT018159_IC_50_HIGH|1170_IC_50_HIGH|HSP90_IC_50_HIGH|None_required_IC_50_HIGH|Y_IC_50_HIGH|0.0390625_IC_50_HIGH,AG-014699_IC_50_LOW|1175_IC_50_LOW|PARP1/2_IC_50_LOW|None_required_IC_50_LOW|Y_IC_50_LOW|0.01953125_IC_50_LOW,AG-014699_IC_50_HIGH|1175_IC_50_HIGH|PARP1/2_IC_50_HIGH|None_required_IC_50_HIGH|Y_IC_50_HIGH|0.01953125_IC_50_HIGH
0,MC-CAR,683665,blood,haematopoietic_neoplasm_other,3.82797,1.49063,2.12056,5.55221,0.801777,-3.76088,...,3.4425,10.2252,2.77958,6.21698,-2.24148,-1.72396,1.51365,2.16016,2.25413,6.31001
1,PFSK-1,683667,nervous_system,medulloblastoma,NaN,NaN,NaN,NaN,NaN,NaN,...,2.62776,9.59003,2.44042,7.07114,-2.17844,-1.29556,2.18165,6.42035,2.34897,8.66015
2,A673,684052,soft_tissue,rhabdomyosarcoma,NaN,NaN,NaN,NaN,NaN,NaN,...,4.01037,11.108,2.58201,4.73163,-5.27223,-0.845975,1.9815,2.59605,1.36835,2.14998
3,ES3,684055,bone,ewings_sarcoma,5.45745,2.71272,3.62435,5.742,1.46516,2.23073,...,1.99072,4.76925,1.72621,3.80154,-1.46239,-0.580209,1.78713,4.72259,0.782539,1.97259
4,ES5,684057,bone,ewings_sarcoma,6.17794,2.71328,3.20822,2.67187,1.48218,2.06263,...,3.37927,8.6078,2.4398,4.02233,-1.21503,0.739059,2.12683,4.04513,1.38483,2.372


In [6]:
# Pivot all measurements into rows, adding in appropriate metadata for each 
# row on the nature of the measurement taken
d_drug = d_sub.set_index(['CellLine', 'CosmicID', 'CancerType', 'Tissue'])
d_drug.columns.name = 'ExperimentDetails'
d_drug = d_drug.stack(dropna=False)
d_drug.name = 'Value'
d_drug = d_drug.reset_index()

# Remove null values in this long format
d_drug = d_drug[d_drug['Value'].notnull()]

# Convert all values to float and ensure that are incapable of conversion
d_drug['Value'] = pd.to_numeric(d_drug['Value'], errors='coerce')
assert d_drug['Value'].isnull().sum() == 0

# Parse drug name and value unit scale out of experiment details
s_rx_id = d_drug['ExperimentDetails'].str.split('|')
d_drug['DrugName'] = s_rx_id.str[0].str.split('_').str[0]
d_drug['ValueUnit'] = s_rx_id.str[0].str.split('_').str[1:].str.join('_')

d_drug.head()

,CellLine,CosmicID,CancerType,Tissue,ExperimentDetails,Value,DrugName,ValueUnit
0,MC-CAR,683665,blood,haematopoietic_neoplasm_other,Erlotinib_IC_50|1_IC_50|EGFR_IC_50|None_requir...,3.827969,Erlotinib,IC_50
1,MC-CAR,683665,blood,haematopoietic_neoplasm_other,Rapamycin_IC_50|3_IC_50|mTOR_IC_50|None_requir...,1.490631,Rapamycin,IC_50
2,MC-CAR,683665,blood,haematopoietic_neoplasm_other,Sunitinib_IC_50|5_IC_50|PDGFRA_:__PDGFRB_:__KD...,2.120559,Sunitinib,IC_50
3,MC-CAR,683665,blood,haematopoietic_neoplasm_other,PHA-665752_IC_50|6_IC_50|MET_IC_50|None_requir...,5.552208,PHA-665752,IC_50
4,MC-CAR,683665,blood,haematopoietic_neoplasm_other,MG-132_IC_50|9_IC_50|Proteosome_IC_50|None_req...,0.801777,MG-132,IC_50


In [7]:
d_drug.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 958836 entries, 0 to 1187759
Data columns (total 8 columns):
CellLine             958836 non-null object
CosmicID             958836 non-null int64
CancerType           958836 non-null object
Tissue               958836 non-null object
ExperimentDetails    958836 non-null object
Value                958836 non-null float64
DrugName             958836 non-null object
ValueUnit            958836 non-null object
dtypes: float64(1), int64(1), object(6)
memory usage: 65.8+ MB


## Export

In [17]:
data.save('prepared', 'response_cosmic', d_drug)

2016-07-08 09:59:30,048:DEBUG:research.project.manager: Saving data to location "/Users/eczech/data/research/musc_genomics/prepared/response_cosmic.pkl"


'/Users/eczech/data/research/musc_genomics/prepared/response_cosmic.pkl'

## Analysis

In [13]:
i_navitoclax = 'ABT-263'
i_other = ['PLX4720', 'SB590885', 'AZD2281', 'AG-014699', 'Nutlin-3a', 'PD-0332991', 'ABT-263']

In [16]:
dt = d_drug[d_drug['DrugName'].isin(i_other)]
dt = dt[dt['ValueUnit'] == 'IC_50']
dt.set_index(['CellLine', 'DrugName'])['Value'].unstack().info()

<class 'pandas.core.frame.DataFrame'>
Index: 666 entries, 22RV1 to no-11
Data columns (total 6 columns):
ABT-263       661 non-null float64
AG-014699     653 non-null float64
Nutlin-3a     661 non-null float64
PD-0332991    633 non-null float64
PLX4720       661 non-null float64
SB590885      641 non-null float64
dtypes: float64(6)
memory usage: 36.4+ KB
